In [ ]:
import gym 
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from robot import Robot
import os

In [ ]:
# load the robot interface gym environment
robot=Robot()

# validate that the robot interface is valid
check_env(robot, warn=True)

In [ ]:
# test the robot interface
episodes = 5
for episode in range(1, episodes+1):
    state = robot.reset()
    done = False
    score = 0 
    
    while not done:
        robot.render()
        action = robot.action_space.sample()
        n_state, reward, done, info = robot.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
robot.close()

In [ ]:
# train the model
log_path = os.path.join(os.path, 'logs')
model = PPO("MlpPolicy", robot, verbose=1, tensorboard_log=log_path)
model.learn(total_timesteps=4000)

In [ ]:
# save the model
model.save('PPO')
evaluate_policy(model, robot, n_eval_episodes=10, render=False)